# Análise exploratória do dataset "Pfizer Vaccine Tweets"
Aqui eu apresento uma análise exploratória simples do dataset "Pfizer Vaccine Tweets" disponível na plataforma Kaggle (https://www.kaggle.com/gpreda/pfizer-vaccine-tweets), que contém diversos tweets com informações complementares sobre os usuários responsáveis por esses tweets. Utilizei a linguagem python juntamente com a biblioteca pandas, cuja documentação na internet é bem explicativa e completa (Link para a documentação: https://pandas.pydata.org/pandas-docs/stable/index.html).

## Carregando biblioteca e dados

In [ ]:
import pandas as pd

In [ ]:
tweets = pd.read_csv('/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv')

print(f'Esse dataset tem {tweets.id.count()} valores')

# Descartando a coluna id pois não nos importa nesse momento
tweets = tweets.drop('id', axis=1)

tweets[5:10]

## Descrevendo as colunas
Aqui observaremos informações básicas como valor mínimo, máximo, média, desvio padrão e distribuição dos valores das colunas do dataset analisado.

Para isso, abaixo utilizo o método `describe()` da biblioteca pandas (Link da documentação: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html).

In [ ]:
tweets.describe()

Podemos observar, na tabela acima, que nenhuma das colunas `user_followers`, `user_friends`, `user_favourites`, `retweets` e `favorites` tem valores nulos (isso pode ser observado pela linha `count`). A que possui valores mais altos é a coluna `user_followers` e a com valores mais baixos é a `retweets` (linha `max`).

#### Analisando `user_followers`
Traz a quantidade de usuários da conta que enviou o tweet em questão.
A média dessa coluna se aproxima de 39.222 (coluna `mean`) com um desvio padrão bem alto, de 347.174(coluna `std`), ou seja, há valores nessa coluna que estão bem distantes da média. Podemos observar também, pela análise de quartis (colunas `25%`, `50%` e `75%` - para entender mais sobre quartis: http://www.portalaction.com.br/estatistica-basica/23-quartis) que 75% dos usuários cujos tweets foram selecionados nesse dataset têm até 2.117 seguidores.

#### Analisando `user_friends`
Acredito que seja o tipo de seguidor que o usuário em questão segue de volta.
Com uma média de 1187 e desvio padrão de 2656, também temos muitos valores distantes da média e 75% dos usuários se concentram na faixa de 1260 "`user_friends`".

#### Analisando `user_favourites`
Pelo que eu pesquisei, essa coluna se refere à antiga feature de favoritar um usuário e receber push notifications quando esse usuário realizar alguma postagem.
Novamente vemos um desvio padrão alto de 38.067 para uma média de 13228. Observamos também que o 3º quartil (coluna `75%`) se encontra com valores de até 9974 "`user_favourites`".

#### Analisando `retweets`
A quantidade de retweets do tweet.
Nessa coluna já vemos um comportamento diferente nos valores, onde, mesmo tendo 15 de desvio padrão para 1,69 de média, 75% dos tweets têm até 1 retweet apenas, com um valor máximo de 678.

#### Analisando `favorites`
Quantidade de favoritos, equivalente a curtidas, do tweet.
Como a coluna anterior, temos um média baixa, igual a 10, mas com desvio padrão alto, equivalente a 70. 75% dos tweets do dataset têm até 5 favoritações apenas, mas o valor máximo encontrado é de 2315.

Abaixo, imprimi gráficos do tipo histograma (https://pt.wikipedia.org/wiki/Histograma). Ele nos ajuda a entender a distribuição dos dados disponíveis dentre os valores daquela coluna. O mesmo poderia ser feito com o gráfico kde (um bom link com explicação de kernel density estimation: https://mathisonian.github.io/kde/), por exemplo, que mostra basicamente um histograma discreto (considerando valores contínuos, e não apenas inteiros). Nos gráficos abaixo temos, no eixo x, os valores da coluna selecionada e, no eixo y, a densidade (ou concentração) de ocorrências para cada valor no eixo x. Para o estudo de cada variável, realizei um filtro que limita os dados aos valores onde temos maior concentração de ocorrências. Busquei realizar esse recorte em valores que estão apenas acima do 3º quartil, assim garantimos que estamos olhando pra pelo menos 75% dos dados totais e removemos apenas os outliers mais distantes.

- Documentação histograma pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html
- Documentação kde pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.kde.html

Nos histogramas abaixo podemos observar os "picos" de concentração em determinados valores das colunas. Por exemplo, na coluna `user_followers` temos uma concentração significativa de tweets cujas contas têm entre 0 e 250 seguidores, sofrendo uma queda brusca conforme o valor da quantidade de seguidores (eixo x) vai crescendo.

In [ ]:
tweets[tweets['user_followers']<3000].user_followers.plot.hist();

In [ ]:
tweets[tweets['user_friends']<7000].user_friends.plot.hist();

Uma observação interessante é que esses valores não necessariamente tem apenas um pico, como podemos observar no aumento da frequência de valores próximos de 5000 `user_friends`.

In [ ]:
tweets[tweets['user_favourites']<50000].user_favourites.plot.hist();

In [ ]:
tweets[tweets['retweets']<20].retweets.plot.hist();

In [ ]:
tweets[tweets['favorites']<100].favorites.plot.hist();

Com os dois gráficos imediatamente acima, esses tweets foram pouco retwitados e pouco favoritados no geral.

### Analisando como as colunas se relacionam
Uma das formas, é utilizando o método `.corr()` do pandas (Link da documentação: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html). 
Em uma rápida explicação, esse método devolve um número de -1 a 1. Quanto mais próximo das extremidades (-1 e 1), mais forte a correlação entre as duas colunas, podendo ela ser negativa, ou positiva, e, quanto mais próximo de zero, mais fraca a correlação.

No snippet de código abaixo, eu selecionei as colunas de número, onde podemos rodar uma operação de correlação entre elas.

In [ ]:
tweets[['retweets', 'user_followers', 'user_friends', 'user_favourites', 'favorites', 'user_verified']].corr()

Podemos observar uma alta correlação entre as colunas `retweets` e `favorites` (0.63).

Abaixo, filtrei os dados com valores de `retweets` e `favoritos` com mais ocorrências e podemos observar no gráfico que temos uma distribuição "diretamente proporcional" entre os valores das duas colunas, ou seja, enquanto o valor de uma cresce, o comum é que o valor da outra cresça proporcionalmente.

In [ ]:
tweets[tweets['retweets']<30][tweets['favorites']<100].plot.scatter(x='retweets', y='favorites');

A segunda correlação mais forte é entre `user_verified` e `user_followers` (0.32).

A terceira, é entre `user_friends` e `user_favourites`, mas ela ainda é meio fraca (0.21).

Abaixo o gráfico de pontos pode nos mostrar que os dados têm valores bem mais dispersos do que o que vimos no gráfico entre `favorites` e `retweets` acima.

In [ ]:
tweets[tweets['user_friends']<3000][tweets['user_favourites']<3000].plot.scatter(x='user_friends', y='user_favourites');

### Analisando colunas não numéricas
Além das colunas numéricas acima, temos também as colunas de data `user_created` e `date`, booleanas (True ou False) `user_verified` e `is_retweet` e a coluna categórica `source`.

#### Analisando `user_created`
Com o gráfico abaixo podemos verificar que `user_created` varia bastante e vai desde 2006 até 2021 com uns picos de criação, mas acredito que não utilizaremos muito essa informação.

In [ ]:
print(f'A informação `user_created` vai de {tweets["user_created"].min()} a {tweets["user_created"].max()}.')

In [ ]:
tweets['user_created_datetime'] = pd.to_datetime(tweets.user_created)
tweets['user_created_date'] = tweets['user_created_datetime'].dt.date
tweets.groupby(['user_created_date']).count().text.plot.line(rot='45');

#### Analisando `date`
Como podemos separar o campo `date` em ano, mês, dia e horário de criação do tweet, achei interessante analisarmos essa informação de diferentes maneiras.

In [ ]:
print(f'A informação `date` vai de {tweets["date"].min()} a {tweets["date"].max()}.')

In [ ]:
tweets['datetime'] = pd.to_datetime(tweets.date)
tweets['year'] = tweets['datetime'].dt.year
tweets['month'] = tweets['datetime'].dt.month
tweets['hour'] = tweets['datetime'].dt.hour
tweets['date'] = tweets['datetime'].dt.date
tweets.groupby(['date']).count().text.plot.line(rot='45');

A distribuição dos nossos dados olhando para `date` nos mostra que a coleta dos dados selecionou tweets de dezembro de 2020 a fevereiro de 2021, sendo que o único mês completo é janeiro de 2021. Alerto também para o fato de que coletamos apenas 10 dias de dados no mês de fevereiro, por isso, no segundo gráfico (quantidade de tweets por mês e ano) podemos observar uma queda tão brusca na quantidade de tweets.

In [ ]:
tweets.groupby(['month', 'year']).count().sort_values(['year']).text.plot.line();

No terceiro e último gráfico vemos um aumento considerável na quantidade de tweets entre 5h e 15h, sendo que o maior pico é no fim da tarde e início da noite, com a quantidade de tweets decrescendo bruscamente a partir das 18h.

In [ ]:
tweets.groupby(['hour']).count().text.plot.line();

#### Analisando `user_verified`
Essa coluna traz para nós a informação se o usuário é verificado ou não. Ou seja, em caso de contas muito grandes ou de pessoas famosas, o Twitter verifica se aquela conta é a oficial e a conta recebe um "selo" de verificada.

No gráfico abaixo podemos observar que temos muito mais tweets de pessoas que não têm conta verificada, ou seja, contas geralmente com um menor alcance.

In [ ]:
tweets.groupby('user_verified').count().plot.barh(y='text')

#### Analisando `is_retweet`
Aqui temos a informação se o tweet é um retweet, ou seja, um "compartilhamento" do tweet original. Todos os tweets do nosso dataset não são retweets.

In [ ]:
tweets.groupby('is_retweet').count().plot.barh(y='text')

#### Analisando `source`
`source` é a coluna que carrega a informação da plataforma de origem do tweet.

Com o gráfico abaixo podemos observar que as três plataformas mais comuns são, nessa ordem:
- Twitter for iPhone
- Twitter Web App
- Twitter for Android

In [ ]:
by_source = tweets.groupby('source').count()
by_source[by_source['text'] > 10].sort_values('text').plot.barh(y='text');

### Analisando colunas textuais
As colunas `user_name`, `user_description`, `user_location`, `text` e `hashtags` são colunas que carregam informações de 
Quando nós, humanos, interpretamos textos, para conseguirmos entender o que está sendo dito precisamos ter pelo menos algum conhecimento do vocabulário utilizado e da relação entre as palavras escritas.
Quando tratamos de análise textual, estamos falando de dados agregados de uma forma não estruturada e com símbolos (letras), que para um computador não têm o mesmo significado que para nós. Para podermos realizar operações como de análise de similaridade e de sentimento textual, precisamos transformar o nosso texto em matrizes numéricas que carreguem informações sobre o significado das palavras e o posicionamento delas no texto. Uma das abordagens 

Acredito que as colunas mais importantes são `text` e `hashtags`, então focarei minhas análises nelas.

#### Analisando `hashtags`
Essa coluna é consideravelmente mais simples do que a text e carrega todas as hashtags citadas no tweet.

In [ ]:
# separando texto com hastags em uma lista de hashtags
tweets_with_hashtags = tweets[tweets.hashtags.notnull()]
tweets_with_hashtags['hashtags_list'] = tweets_with_hashtags.hashtags.apply(lambda x: x[1:-1].split(','))

In [ ]:
# criando um dicionário com a hashtag e a quantidade de ocorrências da mesma
dict_hashtags = dict()
for tweet in tweets_with_hashtags.hashtags_list:
    for hashtag in tweet:
        hashtag = hashtag.strip()
        if hashtag in dict_hashtags.keys():
            dict_hashtags[hashtag] += 1
        else:
            dict_hashtags[hashtag] = 1

dict_hashtags = sorted(dict_hashtags.items(), key=lambda x: x[1], reverse=True)
dict_hashtags[:10]

In [ ]:
import matplotlib.pyplot as plt

plt.bar(*zip(*dict_hashtags[:10]))
plt.xticks(rotation = 45)
plt.show();

No gráfico acima observamos as top 10 hastags que mais aparecem no nosso dataset.

#### Analisando `text`
Essa é a coluna mais importante do dataset. É daqui que tiraremos as informações que realmente queremos encontrar com a análise desse dataset: o que foi twitado sobre a vacina da Pfizer. Construi um pipeline de extração de informações utilizando a biblioteca scikit learn <https://scikit-learn.org> (ou sklearn).

Para a extração de informação desse texto escolhi a 

In [ ]:
from sklearn.feature_extraction import